# Scraping BDC portfolio data from Edgar into useful data
---
Quick iPython notebook to automate the retrieval, cleaning, and processing of a table from a larger SEC filing.

In [1]:
import requests                   
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### Example filing we'll be using - Capital Southwest's 12/13/2017 10-Q

In [2]:
url = 'https://www.sec.gov/Archives/edgar/data/17313/000001731318000007/cswc-20171231x10q.htm'
portfolio = 'CSWC'

#### First - retrieve the html and get the tables out of the html

In [3]:
def get_tables_from_http(url):
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'lxml')
    return soup.find_all('table')

tables = get_tables_from_http(url)

#### Search for the term 'Maturity' - which will definitely be in the tables we need

In [4]:
port_tables = []
for index, table in enumerate(tables):
    if 'Maturity' in table.get_text():
        port_tables.append(index)

# This gives us a list of tables that contain the term 'Maturity' in their text
print(port_tables)

[8, 9, 10, 27, 28, 115, 116, 117, 124, 125, 126]


#### Next, take a look at the tables returned to determine which need to be parsed.... this step is manual

In [5]:
def view_table(tables, table_num):
    temp_code = 'klt_temp'
    x = tables[table_num].text
    y = x.replace('\n\n\n\n\n\n',temp_code)    # these sequence of 6 new lines seems to be unique to their line break
    z = y.replace('\n', '')
    x = z.replace(temp_code,'\n')
    print(x)

# use values from port_tables in 'table_num' to view a prettified version of the data
view_table(tables, 8)         

                   
            Current                        
  Type of   Interest         Fair 
Portfolio Company1 Investment2 Industry Rate3 Maturity Principal Cost Value4 
Non-control/Non-affiliate Investments5                  
AAC HOLDINGS, INC. First Lien Healthcare services L+6.75% (Floor 1.00%), Current Coupon 8.13% 6/30/2023 $ 9,381,250 $ 9,161,050 $ 9,428,156 
AG KINGS HOLDINGS INC.8 First Lien Food, agriculture & beverage L+9.38% (Floor 1.00%), Current Coupon 10.79% 8/8/2021   9,725,000   9,573,185   9,433,250 
ALLIANCE SPORTS GROUP, L.P. Senior subordinated debt Consumer products & retail 11.00% 2/1/2023   10,100,000   9,909,314   9,923,250 
  2.65% membership interest   - -  -   2,500,000   2,500,000 
               12,409,314   12,423,250 
AMERICAN TELECONFERENCING SERVICES, LTD. First Lien Telecommunications L+6.50% (Floor 1.00%), Current Coupon 7.90% 12/8/2021   6,467,005   6,318,452   6,382,126 
  Second Lien   L+9.50% (Floor 1.00%), Current Coupon 10.85% 6/6/2022   

#### Based on our review - use tables 8, 9, 10

In [6]:
port_tables = [8,9,10]
table_html = ''.join([str(tables[i]) for i in port_tables])

#### Read these three tables into data frames, and merge into one larger dataframe

In [7]:
df_html = pd.read_html(str(table_html))
for frame in df_html:
    print(frame.shape)
df = pd.concat(df_html)
df

(25, 19)
(32, 19)
(22, 19)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,Current,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Type of,NaN,NaN,NaN,Interest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fair,NaN,NaN
3,Portfolio Company1,NaN,Investment2,NaN,Industry,NaN,Rate3,NaN,Maturity,NaN,Principal,NaN,Cost,NaN,Value4,NaN,NaN,NaN,NaN
4,Non-control/Non-affiliate Investments5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"AAC HOLDINGS, INC.",NaN,First Lien,NaN,Healthcare services,NaN,"L+6.75% (Floor 1.00%), Current Coupon 8.13%",NaN,6/30/2023,NaN,$,9381250,NaN,$,9161050,NaN,$,9428156,NaN
6,AG KINGS HOLDINGS INC.8,NaN,First Lien,NaN,"Food, agriculture & beverage",NaN,"L+9.38% (Floor 1.00%), Current Coupon 10.79%",NaN,8/8/2021,NaN,NaN,9725000,NaN,NaN,9573185,NaN,NaN,9433250,NaN
7,"ALLIANCE SPORTS GROUP, L.P.",NaN,Senior subordinated debt,NaN,Consumer products & retail,NaN,11.00%,NaN,2/1/2023,NaN,NaN,10100000,NaN,NaN,9909314,NaN,NaN,9923250,NaN
8,NaN,NaN,2.65% membership interest,NaN,NaN,NaN,-,NaN,-,NaN,NaN,-,NaN,NaN,2500000,NaN,NaN,2500000,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12409314,NaN,NaN,12423250,NaN


In [8]:
df = df.drop([1,3,5,7,9,10,12,13,15,16,18], axis=1)     # remove the filler columns
df.columns = ['Issuer', 'Type', 'Industry', 'Rate', 'Maturity', 'Principal', 'Cost', 'Fair Value']
df

,Issuer,Type,Industry,Rate,Maturity,Principal,Cost,Fair Value
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Current,NaN,NaN,NaN,NaN
2,NaN,Type of,NaN,Interest,NaN,NaN,NaN,NaN
3,Portfolio Company1,Investment2,Industry,Rate3,Maturity,NaN,Value4,NaN
4,Non-control/Non-affiliate Investments5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"AAC HOLDINGS, INC.",First Lien,Healthcare services,"L+6.75% (Floor 1.00%), Current Coupon 8.13%",6/30/2023,9381250,9161050,9428156
6,AG KINGS HOLDINGS INC.8,First Lien,"Food, agriculture & beverage","L+9.38% (Floor 1.00%), Current Coupon 10.79%",8/8/2021,9725000,9573185,9433250
7,"ALLIANCE SPORTS GROUP, L.P.",Senior subordinated debt,Consumer products & retail,11.00%,2/1/2023,10100000,9909314,9923250
8,NaN,2.65% membership interest,NaN,-,-,-,2500000,2500000
9,NaN,NaN,NaN,NaN,NaN,NaN,12409314,12423250


In [9]:
# drop the "subtotal" rows by removing rows where 'Type' column is NaN
df.dropna(subset=['Type'], inplace=True)
df = df.reset_index(drop = True)
df

,Issuer,Type,Industry,Rate,Maturity,Principal,Cost,Fair Value
0,NaN,Type of,NaN,Interest,NaN,NaN,NaN,NaN
1,Portfolio Company1,Investment2,Industry,Rate3,Maturity,NaN,Value4,NaN
2,"AAC HOLDINGS, INC.",First Lien,Healthcare services,"L+6.75% (Floor 1.00%), Current Coupon 8.13%",6/30/2023,9381250,9161050,9428156
3,AG KINGS HOLDINGS INC.8,First Lien,"Food, agriculture & beverage","L+9.38% (Floor 1.00%), Current Coupon 10.79%",8/8/2021,9725000,9573185,9433250
4,"ALLIANCE SPORTS GROUP, L.P.",Senior subordinated debt,Consumer products & retail,11.00%,2/1/2023,10100000,9909314,9923250
5,NaN,2.65% membership interest,NaN,-,-,-,2500000,2500000
6,"AMERICAN TELECONFERENCING SERVICES, LTD.",First Lien,Telecommunications,"L+6.50% (Floor 1.00%), Current Coupon 7.90%",12/8/2021,6467005,6318452,6382126
7,NaN,Second Lien,NaN,"L+9.50% (Floor 1.00%), Current Coupon 10.85%",6/6/2022,2005714,1938118,1893735
8,AMWARE FULFILLMENT LLC,First Lien,Distribution,"L+9.50% (Floor 1.00%), Current Coupon 10.99%",5/21/2019,13659722,13426001,13420677
9,"ARGON MEDICAL DEVICES, INC.",Second Lien,Healthcare products,"L+9.50% (Floor 1.00%), Current Coupon 11.07%",6/23/2022,5000000,4885258,5050000


Convert and numbers in an accounting negative format, such as (1,542) to -1542, as well as - to 0

In [10]:
cols_to_convert = ['Principal', 'Cost', 'Fair Value']
df[cols_to_convert]=(df[cols_to_convert].replace( '[-]','0', regex=True).astype(float, errors='ignore'))
df[cols_to_convert]=(df[cols_to_convert].replace( '[\$,)]','', regex=True).replace( '[(]','-', regex=True ))
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,Issuer,Type,Industry,Rate,Maturity,Principal,Cost,Fair Value
0,NaN,Type of,NaN,Interest,NaN,NaN,NaN,NaN
1,Portfolio Company1,Investment2,Industry,Rate3,Maturity,NaN,NaN,NaN
2,"AAC HOLDINGS, INC.",First Lien,Healthcare services,"L+6.75% (Floor 1.00%), Current Coupon 8.13%",6/30/2023,9381250.0,9161050.0,9428156.0
3,AG KINGS HOLDINGS INC.8,First Lien,"Food, agriculture & beverage","L+9.38% (Floor 1.00%), Current Coupon 10.79%",8/8/2021,9725000.0,9573185.0,9433250.0
4,"ALLIANCE SPORTS GROUP, L.P.",Senior subordinated debt,Consumer products & retail,11.00%,2/1/2023,10100000.0,9909314.0,9923250.0
5,NaN,2.65% membership interest,NaN,-,-,0.0,2500000.0,2500000.0
6,"AMERICAN TELECONFERENCING SERVICES, LTD.",First Lien,Telecommunications,"L+6.50% (Floor 1.00%), Current Coupon 7.90%",12/8/2021,6467005.0,6318452.0,6382126.0
7,NaN,Second Lien,NaN,"L+9.50% (Floor 1.00%), Current Coupon 10.85%",6/6/2022,2005714.0,1938118.0,1893735.0
8,AMWARE FULFILLMENT LLC,First Lien,Distribution,"L+9.50% (Floor 1.00%), Current Coupon 10.99%",5/21/2019,13659722.0,13426001.0,13420677.0
9,"ARGON MEDICAL DEVICES, INC.",Second Lien,Healthcare products,"L+9.50% (Floor 1.00%), Current Coupon 11.07%",6/23/2022,5000000.0,4885258.0,5050000.0


Now get rid of extraneous rows and re-index the rows

In [11]:
# drop the remaining non-data rows by removing rows where 'Fair Value' column is NaN
df.dropna(subset=['Fair Value'], inplace=True)
df = df.reset_index(drop = True)
df

,Issuer,Type,Industry,Rate,Maturity,Principal,Cost,Fair Value
0,"AAC HOLDINGS, INC.",First Lien,Healthcare services,"L+6.75% (Floor 1.00%), Current Coupon 8.13%",6/30/2023,9381250.0,9161050.0,9428156.0
1,AG KINGS HOLDINGS INC.8,First Lien,"Food, agriculture & beverage","L+9.38% (Floor 1.00%), Current Coupon 10.79%",8/8/2021,9725000.0,9573185.0,9433250.0
2,"ALLIANCE SPORTS GROUP, L.P.",Senior subordinated debt,Consumer products & retail,11.00%,2/1/2023,10100000.0,9909314.0,9923250.0
3,NaN,2.65% membership interest,NaN,-,-,0.0,2500000.0,2500000.0
4,"AMERICAN TELECONFERENCING SERVICES, LTD.",First Lien,Telecommunications,"L+6.50% (Floor 1.00%), Current Coupon 7.90%",12/8/2021,6467005.0,6318452.0,6382126.0
5,NaN,Second Lien,NaN,"L+9.50% (Floor 1.00%), Current Coupon 10.85%",6/6/2022,2005714.0,1938118.0,1893735.0
6,AMWARE FULFILLMENT LLC,First Lien,Distribution,"L+9.50% (Floor 1.00%), Current Coupon 10.99%",5/21/2019,13659722.0,13426001.0,13420677.0
7,"ARGON MEDICAL DEVICES, INC.",Second Lien,Healthcare products,"L+9.50% (Floor 1.00%), Current Coupon 11.07%",6/23/2022,5000000.0,4885258.0,5050000.0
8,BINSWANGER HOLDING CORP.,First Lien,Distribution,"L+8.00% (Floor 1.00%), Current Coupon 9.49%",3/9/2022,13086113.0,12856165.0,12889821.0
9,NaN,"900,000 shares of common stock",NaN,-,-,0.0,900000.0,874000.0


In [12]:
# also replace '-' with NaN for data uniformity        
df = df.replace({'-':np.nan})
df

,Issuer,Type,Industry,Rate,Maturity,Principal,Cost,Fair Value
0,"AAC HOLDINGS, INC.",First Lien,Healthcare services,"L+6.75% (Floor 1.00%), Current Coupon 8.13%",6/30/2023,9381250.0,9161050.0,9428156.0
1,AG KINGS HOLDINGS INC.8,First Lien,"Food, agriculture & beverage","L+9.38% (Floor 1.00%), Current Coupon 10.79%",8/8/2021,9725000.0,9573185.0,9433250.0
2,"ALLIANCE SPORTS GROUP, L.P.",Senior subordinated debt,Consumer products & retail,11.00%,2/1/2023,10100000.0,9909314.0,9923250.0
3,NaN,2.65% membership interest,NaN,NaN,NaN,0.0,2500000.0,2500000.0
4,"AMERICAN TELECONFERENCING SERVICES, LTD.",First Lien,Telecommunications,"L+6.50% (Floor 1.00%), Current Coupon 7.90%",12/8/2021,6467005.0,6318452.0,6382126.0
5,NaN,Second Lien,NaN,"L+9.50% (Floor 1.00%), Current Coupon 10.85%",6/6/2022,2005714.0,1938118.0,1893735.0
6,AMWARE FULFILLMENT LLC,First Lien,Distribution,"L+9.50% (Floor 1.00%), Current Coupon 10.99%",5/21/2019,13659722.0,13426001.0,13420677.0
7,"ARGON MEDICAL DEVICES, INC.",Second Lien,Healthcare products,"L+9.50% (Floor 1.00%), Current Coupon 11.07%",6/23/2022,5000000.0,4885258.0,5050000.0
8,BINSWANGER HOLDING CORP.,First Lien,Distribution,"L+8.00% (Floor 1.00%), Current Coupon 9.49%",3/9/2022,13086113.0,12856165.0,12889821.0
9,NaN,"900,000 shares of common stock",NaN,NaN,NaN,0.0,900000.0,874000.0


In [13]:
# fill in missing security names and industry
for x in range(df.shape[0]):
    if isinstance(df.loc[x,'Issuer'], str) == False:
        df.loc[x,'Issuer'] = df.loc[x-1,"Issuer"]
        df.loc[x,'Industry'] = df.loc[x-1,"Industry"]
df

,Issuer,Type,Industry,Rate,Maturity,Principal,Cost,Fair Value
0,"AAC HOLDINGS, INC.",First Lien,Healthcare services,"L+6.75% (Floor 1.00%), Current Coupon 8.13%",6/30/2023,9381250.0,9161050.0,9428156.0
1,AG KINGS HOLDINGS INC.8,First Lien,"Food, agriculture & beverage","L+9.38% (Floor 1.00%), Current Coupon 10.79%",8/8/2021,9725000.0,9573185.0,9433250.0
2,"ALLIANCE SPORTS GROUP, L.P.",Senior subordinated debt,Consumer products & retail,11.00%,2/1/2023,10100000.0,9909314.0,9923250.0
3,"ALLIANCE SPORTS GROUP, L.P.",2.65% membership interest,Consumer products & retail,NaN,NaN,0.0,2500000.0,2500000.0
4,"AMERICAN TELECONFERENCING SERVICES, LTD.",First Lien,Telecommunications,"L+6.50% (Floor 1.00%), Current Coupon 7.90%",12/8/2021,6467005.0,6318452.0,6382126.0
5,"AMERICAN TELECONFERENCING SERVICES, LTD.",Second Lien,Telecommunications,"L+9.50% (Floor 1.00%), Current Coupon 10.85%",6/6/2022,2005714.0,1938118.0,1893735.0
6,AMWARE FULFILLMENT LLC,First Lien,Distribution,"L+9.50% (Floor 1.00%), Current Coupon 10.99%",5/21/2019,13659722.0,13426001.0,13420677.0
7,"ARGON MEDICAL DEVICES, INC.",Second Lien,Healthcare products,"L+9.50% (Floor 1.00%), Current Coupon 11.07%",6/23/2022,5000000.0,4885258.0,5050000.0
8,BINSWANGER HOLDING CORP.,First Lien,Distribution,"L+8.00% (Floor 1.00%), Current Coupon 9.49%",3/9/2022,13086113.0,12856165.0,12889821.0
9,BINSWANGER HOLDING CORP.,"900,000 shares of common stock",Distribution,NaN,NaN,0.0,900000.0,874000.0


# To do
** much better documentation ** 
1. Strip out footnotes from company names
1. Split out the rate
2. Add debt/equity column
3. Do summary statistics

## Next step
1. Export to SQL